In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import os
from pprint import pprint

# Importing the models to be tested
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier, Pool


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from auxiliar_func import *
from plot_func import *

In [2]:
target = 'income_50k'
df_tr = pd.read_csv('train.csv')

TARGET_METRIC = 'f1_macro'
SEED = 42

prep_params_grid = {
    'scaling': [None, 'minmax', 'standard'],
    'imputation': ['mode'],
    'cat_age': [False, True],
    'merge_capital': [False, True],
    'downsampling_method': ['random'],
    'target_freq': [0.75, 0.8, 0.85],
    'generate_dummies': [True]
}

def n_comb(grid: dict, print_=True):
    n = 1
    for k in grid.keys():
        n *= len(grid[k])
    if print_:
        print(f'Number of model combinations to be tested: {n}')
    else:
        return n

def test_model(mod, prep_grid, mod_grid, name, rewrite=False, prev_res=False, **kwargs):
    if rewrite or not os.path.exists(f'./results/results_{name}.csv'):
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore") # to avoid convergence warnings
                results = search_best_combination(mod, mod_grid, prep_grid, df_tr, target_metric=TARGET_METRIC, cv=5, **kwargs)
                results.to_csv(f'./results/results_{name}.csv', index=False)
        except Exception as e:
            print(e)
            return None

n_comb(prep_params_grid)

Number of model combinations to be tested: 36


### Tuning parameters for LDA

In [3]:
mod_par_grid = {
    'solver': ['svd', 'lsqr', 'eigen'],
    'shrinkage': [None, 'auto'],
    'store_covariance': [True, False],
    'tol': [1e-4, 1e-3, 1e-2],
}

n_comb(mod_par_grid)

lda = LDA()
test_model(lda, prep_params_grid, mod_par_grid, 'lda')

Number of model combinations to be tested: 36


### Tuning parameters for QDA

In [4]:
mod_par_grid = {
    'reg_param': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'store_covariance': [True, False],
    'tol': [1e-4, 1e-3, 1e-2],
}

n_comb(mod_par_grid)

qda = QDA()
test_model(qda, prep_params_grid, mod_par_grid, 'qda')

Number of model combinations to be tested: 36


### Tuning parameters for KNN

In [5]:
mod_par_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
}

n_comb(mod_par_grid)

knn = KNN()
test_model(knn, prep_params_grid, mod_par_grid, 'knn')

Number of model combinations to be tested: 14


### Tuning parameters for logistic regression

In [6]:
mod_par_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1, 10],
    'class_weight': [None, 'balanced'],
    'fit_intercept': [True, False],
    'intercept_scaling': [0.1, 1, 10],
    'max_iter': [1000],
    'multi_class': ['auto'],
    'random_state': [SEED],
    'solver': ['saga']
}

n_comb(mod_par_grid)

logreg = LogisticRegression()
test_model(logreg, prep_params_grid, mod_par_grid, 'logreg')

Number of model combinations to be tested: 72


### Tuning parameters for SVM

In [7]:
mod_par_grid = {
    'penalty': ['l1', 'l2'],
    'loss': ['squared_hinge','hinge'],
    'dual': [False],
    'C': [0.1, 1, 10],
    'fit_intercept': [True, False],
    'intercept_scaling': [0.1, 1],
    'class_weight': [None, 'balanced'],
    'max_iter': [1000],
    'random_state': [SEED]
}

n_comb(mod_par_grid)

svm = LinearSVC()
test_model(svm, prep_params_grid, mod_par_grid, 'svm')

Number of model combinations to be tested: 96


### Tuning parameters for random forest

In [8]:
mod_par_grid = {
    'n_estimators': [50, 65, 75, 100],
    'criterion': ['gini'],
    'max_depth': [None, 25, 30, 35, 40],
    'max_features': ['sqrt', 'log2'],
    'random_state': [SEED],
    'verbose': [0],
    'warm_start': [False],
    'class_weight': [None, 'balanced']
}

n_comb(mod_par_grid)

rf = RandomForestClassifier()
test_model(rf, prep_params_grid, mod_par_grid, 'rf')

Number of model combinations to be tested: 80


### Tuning parameters for catboost

In [9]:
prep_params_grid2 = {
    'scaling': [None],
    'imputation': ['mode'],
    'cat_age': [False],
    'merge_capital': [False, True],
    'downsampling_method': ['random'],
    'target_freq': [0.8, 0.85, 0.9],
    'generate_dummies': [False]
}

# a first preprocess to get the categorical features
df_tr_pre = preprocessing(df_tr, imputation='mode', cat_age=False, generate_dummies=False)
X_train, y_train = df_tr_pre.drop(target, axis=1), df_tr_pre[target]
cat_features = list(X_train.select_dtypes(include=['category']).columns)

mod_par_grid = {
    'iterations': [750],
    'depth': [4, 6, 8],
    'border_count': [32, 64, 96],
    'random_seed': [SEED],
    'verbose': [0],
    'loss_function': ['Logloss'],
    'eval_metric': ['F1', 'AUC'],
    'class_weights': [[1, 1], [1, 2]],
    'cat_features': [cat_features],
}

n_comb(mod_par_grid)

cat_model = CatBoostClassifier()
test_model(cat_model, prep_params_grid2, mod_par_grid, 'catboost', rewrite=True)

Number of model combinations to be tested: 36
===Iteration 1===
Searching preprocessing parameters...
it: 6/6
Searching model parameters...
it: 36/36
Best metric: 0.79839712741321
Best preprocessing parameters: [{'scaling': None, 'imputation': 'mode', 'cat_age': False, 'merge_capital': False, 'downsampling_method': 'random', 'target_freq': 0.85, 'generate_dummies': False, 'remove_duplicates': True}, {'scaling': None, 'imputation': 'mode', 'cat_age': False, 'merge_capital': True, 'downsampling_method': 'random', 'target_freq': 0.85, 'generate_dummies': False, 'remove_duplicates': True}, {'scaling': None, 'imputation': 'mode', 'cat_age': False, 'merge_capital': False, 'downsampling_method': 'random', 'target_freq': 0.8, 'generate_dummies': False, 'remove_duplicates': True}, {'scaling': None, 'imputation': 'mode', 'cat_age': False, 'merge_capital': True, 'downsampling_method': 'random', 'target_freq': 0.8, 'generate_dummies': False, 'remove_duplicates': True}, {'scaling': None, 'imputatio